<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 75:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.325763702392578

Current batch: 100
Current benign train accuracy: 0.3125
Current benign train loss: 1.9976322650909424

Current batch: 200
Current benign train accuracy: 0.3671875
Current benign train loss: 1.9082565307617188

Current batch: 300
Current benign train accuracy: 0.3125
Current benign train loss: 1.8486440181732178

Current batch: 400
Current benign train accuracy: 0.421875
Current benign train loss: 1.7275234460830688

Current batch: 500
Current benign train accuracy: 0.28125
Current benign train loss: 1.8894047737121582

Current batch: 600
Current benign train accuracy: 0.4453125
Current benign train loss: 1.6072123050689697

Current batch: 700
Current benign train accuracy: 0.4140625
Current benign train loss: 1.5376538038253784

Current batch: 800
Current benign train accuracy: 0.3828125
Current benign train loss: 1.6975250244140625

Current batch: 900
Current b


Current batch: 800
Current benign train accuracy: 0.734375
Current benign train loss: 0.7282472848892212

Current batch: 900
Current benign train accuracy: 0.765625
Current benign train loss: 0.6749134063720703

Current batch: 1000
Current benign train accuracy: 0.8046875
Current benign train loss: 0.7047449946403503

Current batch: 1100
Current benign train accuracy: 0.78125
Current benign train loss: 0.6153210401535034

Total benign train accuarcy: 75.71
Total benign train loss: 817.1468981802464

[ Test epoch: 5 ]

Test accuarcy: 75.89
Test average loss: 0.007094757473468781
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.765625
Current benign train loss: 0.5799041390419006

Current batch: 100
Current benign train accuracy: 0.828125
Current benign train loss: 0.5923219919204712

Current batch: 200
Current benign train accuracy: 0.8359375
Current benign train loss: 0.5763957500457764

Current batch: 300
Current benign train accuracy: 0.7890625
Cur


Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.42430058121681213

Current batch: 300
Current benign train accuracy: 0.8125
Current benign train loss: 0.5136774778366089

Current batch: 400
Current benign train accuracy: 0.7109375
Current benign train loss: 0.7049401998519897

Current batch: 500
Current benign train accuracy: 0.78125
Current benign train loss: 0.6432186961174011

Current batch: 600
Current benign train accuracy: 0.8046875
Current benign train loss: 0.4976898431777954

Current batch: 700
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5325759649276733

Current batch: 800
Current benign train accuracy: 0.8125
Current benign train loss: 0.581158459186554

Current batch: 900
Current benign train accuracy: 0.828125
Current benign train loss: 0.4882718026638031

Current batch: 1000
Current benign train accuracy: 0.796875
Current benign train loss: 0.6103342771530151

Current batch: 1100
Current benign train accu


Current batch: 1000
Current benign train accuracy: 0.765625
Current benign train loss: 0.6193283796310425

Current batch: 1100
Current benign train accuracy: 0.84375
Current benign train loss: 0.4314916133880615

Total benign train accuarcy: 83.83533333333334
Total benign train loss: 544.8421967327595

[ Test epoch: 16 ]

Test accuarcy: 79.84
Test average loss: 0.006063683465123176
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.8359375
Current benign train loss: 0.44329142570495605

Current batch: 100
Current benign train accuracy: 0.8359375
Current benign train loss: 0.4510117471218109

Current batch: 200
Current benign train accuracy: 0.890625
Current benign train loss: 0.4013173282146454

Current batch: 300
Current benign train accuracy: 0.8359375
Current benign train loss: 0.43569546937942505

Current batch: 400
Current benign train accuracy: 0.890625
Current benign train loss: 0.3931114375591278

Current batch: 500
Current benign train accura


Current batch: 400
Current benign train accuracy: 0.875
Current benign train loss: 0.3889332711696625

Current batch: 500
Current benign train accuracy: 0.84375
Current benign train loss: 0.46706920862197876

Current batch: 600
Current benign train accuracy: 0.859375
Current benign train loss: 0.39581364393234253

Current batch: 700
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3962278962135315

Current batch: 800
Current benign train accuracy: 0.828125
Current benign train loss: 0.47732484340667725

Current batch: 900
Current benign train accuracy: 0.8671875
Current benign train loss: 0.3828985095024109

Current batch: 1000
Current benign train accuracy: 0.8203125
Current benign train loss: 0.41882070899009705

Current batch: 1100
Current benign train accuracy: 0.84375
Current benign train loss: 0.40465405583381653

Total benign train accuarcy: 85.32666666666667
Total benign train loss: 495.50507389009

[ Test epoch: 22 ]

Test accuarcy: 74.66
Test average los

...


Current batch: 500
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08789850771427155

Current batch: 600
Current benign train accuracy: 0.953125
Current benign train loss: 0.13838931918144226

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.08299104869365692

Current batch: 800
Current benign train accuracy: 0.9765625
Current benign train loss: 0.1289270967245102

Current batch: 900
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09582400321960449

Current batch: 1000
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1699162721633911

Current batch: 1100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.1400078386068344

Total benign train accuarcy: 96.74466666666666
Total benign train loss: 114.40574619546533

[ Test epoch: 72 ]

Test accuarcy: 87.66
Test average loss: 0.004806440473347902
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train a


Total benign train accuarcy: 98.48466666666667
Total benign train loss: 56.79586466494948

[ Test epoch: 77 ]

Test accuarcy: 91.08
Test average loss: 0.0032455664858222007
Model Saved!

[ Train epoch: 78 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.01795891486108303

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.025213629007339478

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.027684666216373444

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.04364469274878502

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.022624783217906952

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.02154148928821087

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.039558202028274536

Current batch: 700
Current benign train accurac


Current batch: 500
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07656104117631912

Current batch: 600
Current benign train accuracy: 0.984375
Current benign train loss: 0.03291909769177437

Current batch: 700
Current benign train accuracy: 0.984375
Current benign train loss: 0.03347266837954521

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.044444113969802856

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.013410178944468498

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.01083133090287447

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.011600387282669544

Total benign train accuarcy: 98.69133333333333
Total benign train loss: 48.05407319543883

[ Test epoch: 83 ]

Test accuarcy: 91.13
Test average loss: 0.0032842180110514165
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.


Test accuarcy: 91.32
Test average loss: 0.0032796536274254324
Model Saved!

[ Train epoch: 89 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.04651796817779541

Current batch: 100
Current benign train accuracy: 0.984375
Current benign train loss: 0.06503156572580338

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02477165311574936

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02298559620976448

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02463618665933609

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.020945381373167038

Current batch: 600
Current benign train accuracy: 0.9765625
Current benign train loss: 0.040791336447000504

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02593388222157955

Current batch: 800
Current benign train a


Current batch: 600
Current benign train accuracy: 0.9765625
Current benign train loss: 0.043103668838739395

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03529515489935875

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.017928391695022583

Current batch: 900
Current benign train accuracy: 0.9921875
Current benign train loss: 0.017219431698322296

Current batch: 1000
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03543639928102493

Current batch: 1100
Current benign train accuracy: 0.96875
Current benign train loss: 0.07783820480108261

Total benign train accuarcy: 98.942
Total benign train loss: 39.99438986927271

[ Test epoch: 94 ]

Test accuarcy: 91.19
Test average loss: 0.003319209534674883
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.04214772209525108

Current batch: 100
Current benign train accurac